<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/nsclc-radiomics/nsclc_radiomics_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Rationale </center> 

This notebook stores a demo for Hosny et Al. [Deep learning for lung cancer prognostication: A retrospective multi-cohort radiomics study](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1002711), reproduced using the tools provided by the Imaging Data Commons.

<br>

The goal of this notebook is to provide the user with an example of how the tools provided by the Imaging Data Commons could be used to run an AI/ML end-to-end analysis on a cohort hosted by the portal, and to describe what we identified as the best practices to do so.

<br>


---
---

# <center> Environment Setup, Data Download and Pre-processing </center>

In [1]:
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print("Current directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

Current directory : /content
Hostname          : 02824e1fa3e7
Username          : root


## Environment Set Up

This demo notebook was conceived to be run using a GPU.

To access a free GPU on Colab:
`Edit > Notebooks Settings`

From the dropdown menu under `Hardware accelerator`, select `GPU`.

In [2]:
# check wether the use of a GPU was correctly enabled
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

has_gpu

True

---

## Preliminary Notes

Hosny et Al. model was developed using Keras 1.2.2 and an old version of Tensorflow, as stated by the authors (e.g., see [the docker config file in the model GitHub repository](https://github.com/modelhub-ai/deep-prognosis/blob/master/dockerfiles/keras:1.0.1)). Since Google Colab instances are running either TensorFlow 2.x.x or TensorFlow 1.15.2, and Keras 2.x.x, pulling the model from the [project repository](https://github.com/modelhub-ai/deep-prognosis) will not work out-of-the-box (due to compatibility issues between Keras 1.x.x and Keras 2.x.x). 

<br>

While it is possible to [use the `%tensorflow_version 1.x` magic to switch to the latest 1.x version of TensorFlow](https://colab.research.google.com/notebooks/tensorflow_version.ipynb)<sup>*</sup>, Colab does not allow to switch to older version of Keras. Fortunately, the solution to this issue is known and discussed in [various threads](https://github.com/keras-team/keras/issues/6382#issuecomment-530258501). Together with this notebook we hence provide a [Keras-2-compatible network configuration JSON file](https://github.com/ImagingDataCommons/IDC-Examples/blob/master/notebooks/nsclc-radiomics/demo/architecture.json).

<sup>*</sup> the magic command must be run before importing TensorFlow.

<br>

Such JSON file is stored, together with the source code and the files needed for this demo, under the [Imagin Data Common's "IDC-Examples" repository](https://github.com/ImagingDataCommons/IDC-Examples) (specifically, under [IDC-Examples/notebooks/nsclc-radiomics/](https://github.com/ImagingDataCommons/IDC-Examples/tree/master/notebooks/nsclc-radiomics). All the other 

Since we are interested in cloning only a subdirectory of the repository, and the git CLI does not allow that, we can use either [GitHub's CLI](https://cli.github.com/manual/) `gh` (requires authentication) or Apache's subversion `svn` [[1]](https://cheatography.com/davechild/cheat-sheets/subversion/) [[2]](https://stackoverflow.com/questions/7106012/download-a-single-folder-or-directory-from-a-github-repo):

In [3]:
%%capture
!sudo apt install subversion

In [4]:
!svn checkout https://github.com/ImagingDataCommons/IDC-Examples/trunk/notebooks/nsclc-radiomics/demo

A    demo/__init__.py
A    demo/architecture.json
A    demo/data_utils.py
A    demo/requirements.txt
A    demo/utils.py
A    demo/weights.h5
Checked out revision 39.


In [5]:
!pip3 install -r demo/requirements.txt

     |████████████████████████████████| 44.9MB 69kB/s 
     |████████████████████████████████| 35.5MB 75kB/s 


For image pre-processing we will use [Plastimatch](https://plastimatch.org), a reliable and open source software for image computation.

Plastimatch is available as an extension (plug-in) for 3D Slicer, but can also be used from the command line/from python scripts (using libraries such as `subprocess`):

In [6]:
%%capture
!sudo apt update
!sudo apt install plastimatch

Verify the installation process was successful by checking Plastimatch version:

In [7]:
!plastimatch --version

plastimatch version 1.7.0


---

In [9]:
import os
import sys
import json
import numpy as np
import pandas as pd
import SimpleITK as sitk

from IPython.display import clear_output

%tensorflow_version 1.x
import tensorflow as tf
import keras

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)
print("TensorFlow version           : ", tf.__version__)
print("Keras (stand-alone) version  : ", keras.__version__)

print("\nThis Colab instance is equipped with a GPU.")

# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the CT)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:,-1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.jet
my_jet = cmap(np.arange(cmap.N))
my_jet[:,-1] = np.linspace(0, 1, cmap.N)
my_jet = ListedColormap(my_jet)

## ----------------------------------------

import seaborn as sns

Python version               :  3.6.9 (default, Jul 17 2020, 12:50:27) 
Numpy version                :  1.18.5
TensorFlow version           :  1.15.2
Keras (stand-alone) version  :  2.3.1

This Colab instance is equipped with a GPU.


---

## Data Download

The Imaging Data Commons GCS buckets are "[requester pays](https://cloud.google.com/storage/docs/requester-pays)" buckets. Hence, it is not possible to [mount such buckets directly in Colab](https://gist.github.com/korakot/f3600576720206363c734eca5f302e38).

Instead, what the user can do is to query the BigQuery table associated to the bucket/dataset, select the cohort of interest, and then download the files exploiting `gsutil`.

<font size="5" color="orange"><b>This step (auth) won't be needed once the IDC bucket goes live</b></font>

In [ ]:
from google.colab import auth
auth.authenticate_user()

---
#### Exploiting BigQuery to Select which Data to Download

Let's run a BigQuery query, exploiting the `%%bigquery` [IPython Magic](https://googleapis.dev/python/bigquery/latest/magics.html), to parse information regarding the dataset of interest (e.g., which subjects to download, the mapping between DICOM CTs and DICOM RTSTRUCTs/RTSEGs, ...).

Using the following syntax, the result will be stored as a DataFrame in `cohort_df`.

In [ ]:
%%bigquery --project=idc-sandbox-000 cohort_df

WITH
  ct_series AS (
  SELECT
    DISTINCT(PatientID),
    StudyInstanceUID AS ctStudyInstanceUID,
    SeriesInstanceUID AS ctSeriesInstanceUID
  FROM
    `idc-dev-etl.idc_tcia.idc_tcia`
  WHERE
    PatientID LIKE "LUNG1%"
    AND Modality = "CT"
  ORDER BY
    PatientID),
  rtstruct_series AS (
  SELECT
    DISTINCT(PatientID),
    StudyInstanceUID AS rtstructStudyInstanceUID,
    SeriesInstanceUID AS rtstructSeriesInstanceUID
  FROM
    `idc-dev-etl.idc_tcia.idc_tcia`
  WHERE
    PatientID LIKE "LUNG1%"
    AND Modality = "RTSTRUCT"
  ORDER BY
    PatientID),
  seg_series AS (
  SELECT
    DISTINCT(PatientID),
    StudyInstanceUID AS segStudyInstanceUID,
    SeriesInstanceUID AS segSeriesInstanceUID
  FROM
    `idc-dev-etl.idc_tcia.idc_tcia`
  WHERE
    PatientID LIKE "LUNG1%"
    AND Modality = "SEG"
  ORDER BY
    PatientID)
SELECT
  PatientID,
  ctStudyInstanceUID,
  ctSeriesInstanceUID,
  rtstructStudyInstanceUID,
  rtstructSeriesInstanceUID,
  segStudyInstanceUID,
  segSeriesInstanceUID
FROM
  ct_series
JOIN
  rtstruct_series
using (PatientID)
JOIN
  seg_series
USING
  (PatientID)
ORDER BY
  PatientID

In [ ]:
cohort_df.head()

After selecting a few subjects from the cohort, exploiting the output of the BigQuery query, populate a dictionary with the `ctStudyInstanceUID` and `rtstructStudyInstanceUID` values. Finally, exploit `gsutil` to download such sub-cohort:

In [ ]:
n_patients = 10

# useful for logging purposes
download_dict = dict()

# since gsutil can be called with "-m" providing a list as an input,
# append each gs URI to download to a list:
to_download = list()
base_gs_uri = 'gs://idc-tcia-1-nsclc-radiomics/dicom/'

subcohort_df = cohort_df.sample(n = n_patients)

for pat_num, pat in enumerate(list(subcohort_df["PatientID"])):
  print("(%g/%g) - PatientID: %s"%(pat_num + 1, n_patients, pat), end = '\r')

  pat_df = subcohort_df[subcohort_df["PatientID"] == pat]
  download_dict[pat] = dict()
  download_dict[pat]["ctStudyInstanceUID"] = pat_df["ctStudyInstanceUID"].values[0]
  download_dict[pat]["rtstructStudyInstanceUID"] = pat_df["rtstructStudyInstanceUID"].values[0]

  to_download.append(base_gs_uri + download_dict[pat]["ctStudyInstanceUID"])
  to_download.append(base_gs_uri + download_dict[pat]["rtstructStudyInstanceUID"])

In [ ]:
# populate a dataframe with such gs URI (again, mostly for logging purposes)
manifesto_dict = {"gs_uri" : to_download}
manifesto_df = pd.DataFrame(manifesto_dict, columns = ["gs_uri"])

manifesto_df.head()

In [ ]:
# generate a text file, which will be parsed by gsutil to download the selected patients:
manifesto_df.to_csv("gcs_paths.txt", header = False, index = False)

# check everything went as expected
!head /content/gcs_paths.txt -n 2
!echo "..." && echo ""

# the number of lines in the file should be equal to twice the specified "n_patients"
!echo "Number of lines in the file:" $(cat /content/gcs_paths.txt | wc -l)

---

#### DICOM Data Download

The following instructions will download the DICOM CT and DICOM RTSTRUCT files for `n_patients` patients. For `n_patients = 10`, the download should take approximately 3-4 minutes (roughly 1-1.5k files, a total of 500-700MB).

In [ ]:
# if everything is allright, proceed with the download
!mkdir -p data/nsclc-radiomics/dicom

!cat gcs_paths.txt | gsutil -u idc-sandbox-000 -m cp -Ir ./data/nsclc-radiomics/dicom

---

Check the disk space after the download:

In [ ]:
!df -h

In [ ]:
!du -h ./data/nsclc-radiomics -d 0


---

## Data Pre-processing

For the time being, copy the python files from the src directory under `My Drive/Colab Notebooks`.

When the specific IDC repository becomes available, just clone that (or, better, [a subdirectory only](https://stackoverflow.com/questions/600079/git-how-do-i-clone-a-subdirectory-only-of-a-git-repository).)

In [ ]:
!ls $drive_src_path

In [ ]:
# replace the following instructions with "git clone" (and the --filter flag?)
# possibly, move this to the top (as for now we need to mount the GDrive folder
# before doing anything, but in the future this will change)
drive_src_path = os.path.join(mount_point, "My\ Drive/Colab\ Notebooks/src/")
!mkdir -p src
!cp -r $drive_src_path .

from src.data_utils import *
from src.utils import *

# FIXME: DEBUG

import importlib
importlib.reload(data_utils)

In [ ]:
data_base_path = 'data'

dataset_name = 'nsclc-radiomics'
dataset_path = os.path.join(data_base_path, dataset_name)
data_path = os.path.join(dataset_path, 'dicom')

preproc_dataset_name = dataset_name + '_preprocessed'
preproc_dataset_path = os.path.join(data_base_path, preproc_dataset_name)
preproc_data_path = os.path.join(preproc_dataset_path, 'nrrd')
    
if not os.path.exists(preproc_data_path):
    os.makedirs(preproc_data_path)

In [ ]:
for pat_num, pat in enumerate(subcohort_df.PatientID.values):

    # clear cell output before moving to the next (goes at the top to clean what comes next)
    clear_output(wait = True)
    
    print("\nPatient %d/%d (%s)"%(pat_num + 1, len(subcohort_df), pat))
    
    pat_dir_path = os.path.join(preproc_data_path, pat)
    
    if not os.path.exists(pat_dir_path):
        os.mkdir(pat_dir_path)
    
    # location where the tmp nrrd files (resampled CT/RTSTRUCT nrrd) should be saved
    # by the "export_res_nrrd_from_dicom" function found in preprocess.py
    ct_nrrd_path = os.path.join(pat_dir_path, pat + '_ct_resampled.nrrd')
    rt_nrrd_path = os.path.join(pat_dir_path, pat + '_rt_resampled.nrrd')

    # location where the nrrd files (cropped resampled CT/RTSTRUCT nrrd) should be saved
    # by the "export_com_subvolume" function found in preprocess.py
    ct_nrrd_crop_path = os.path.join(pat_dir_path, pat + '_ct_res_crop.nrrd')
    rt_nrrd_crop_path = os.path.join(pat_dir_path, pat + '_rt_res_crop.nrrd')
    
    # if the latter are already there, skip the processing
    if os.path.exists(ct_nrrd_crop_path) and os.path.exists(rt_nrrd_crop_path):
        print("%s\nand\n%s\nfound, skipping the processing for patient %s..."%(ct_nrrd_crop_path,
                                                                               rt_nrrd_crop_path, 
                                                                               pat))
        continue
    
    ## ----------------------------------------
    
    pat_df = subcohort_df[subcohort_df["PatientID"] == pat]


    path_to_ct_dir = os.path.join(data_path,
                                  pat_df["ctStudyInstanceUID"].values[0],
                                  pat_df["ctSeriesInstanceUID"].values[0])

    path_to_rt_dir = os.path.join(data_path,
                                  pat_df["rtstructStudyInstanceUID"].values[0],
                                  pat_df["rtstructSeriesInstanceUID"].values[0])

    path_to_seg_dir = os.path.join(data_path, 
                                   pat_df["segStudyInstanceUID"].values[0], 
                                   pat_df["segSeriesInstanceUID"].values[0])

    # FIXME: sanity check
    assert os.path.exists(path_to_ct_dir)
    assert os.path.exists(path_to_rt_dir)
    assert os.path.exists(path_to_seg_dir)    
    
    # log lookup informations (human-readable to StudyUID and SeriesUID)
    lookup_dict_path = os.path.join(pat_dir_path, pat + '_lookup_info.json')
    
    lookup_dict = dict()
    lookup_dict[pat] = dict()
    
    lookup_dict[pat]["path_to_ct_dir"] = path_to_ct_dir
    lookup_dict[pat]["ctStudyInstanceUID"] = pat_df["ctStudyInstanceUID"].values[0]
    lookup_dict[pat]["ctSeriesInstanceUID"] = pat_df["ctSeriesInstanceUID"].values[0]
    
    lookup_dict[pat]["path_to_rt_dir"] = path_to_rt_dir
    lookup_dict[pat]["rtstructStudyInstanceUID"] = pat_df["rtstructStudyInstanceUID"].values[0]
    lookup_dict[pat]["rtstructSeriesInstanceUID"] = pat_df["rtstructSeriesInstanceUID"].values[0]
    
    # FIXME: not used so far but still, useful to log for future purposes
    lookup_dict[pat]["path_to_seg_dir"] = path_to_seg_dir
    lookup_dict[pat]["segStudyInstanceUID"] = pat_df["segStudyInstanceUID"].values[0]
    lookup_dict[pat]["segSeriesInstanceUID"] = pat_df["segSeriesInstanceUID"].values[0]
    
    with open(lookup_dict_path, 'w') as json_file:
        json.dump(lookup_dict, json_file, indent = 2)
    
    ## ----------------------------------------

    proc_log = export_res_nrrd_from_dicom(dicom_ct_path = path_to_ct_dir, 
                                          dicom_rt_path = path_to_rt_dir, 
                                          output_dir = pat_dir_path, 
                                          pat_id = pat,
                                          output_dtype = "float")
    
    # check every step of the DICOM to NRRD conversion returned 0 (everything's ok)
    assert(np.sum(np.array(list(proc_log.values()))) == 0)
    
    # FIXME: sanity check
    assert(os.path.exists(ct_nrrd_path))
    assert(os.path.exists(rt_nrrd_path))
    
    sitk_vol = sitk.ReadImage(ct_nrrd_path)
    vol = sitk.GetArrayFromImage(sitk_vol)
    
    sitk_seg = sitk.ReadImage(rt_nrrd_path)
    seg = sitk.GetArrayFromImage(sitk_seg)
    
    # FIXME: sanity check
    assert(vol.shape == seg.shape)
    
    com = compute_center_of_mass(seg)
    com_int = [int(coord) for coord in com]

    # export the CoM slice (CT + RTSTRUCT) for quality control
    export_png_slice(input_volume = vol,
                     input_segmask = seg,
                     fig_out_path = os.path.join(pat_dir_path, pat + '_whole_CT_CoM.png'),
                     fig_dpi = 220,
                     lon_slice_idx = com_int[0],
                     cor_slice_idx = com_int[1],
                     sag_slice_idx = com_int[2],
                     z_first = True)
    
    # crop a (150, 150, 150) subvolume from the resampled scans, get rid of the latter
    proc_log = export_com_subvolume(ct_nrrd_path = ct_nrrd_path, 
                                    rt_nrrd_path = rt_nrrd_path, 
                                    crop_size = (150, 150, 150), 
                                    output_dir = pat_dir_path,
                                    pat_id = pat,
                                    z_first = True, 
                                    rm_orig = True)
    
    # log CoM information
    com_log_path = os.path.join(pat_dir_path, pat + '_com_log.json')
    com_log_dict = {k : v for (k, v) in proc_log.items() if "com_int" in k}
    
    with open(com_log_path, 'w') as json_file:
        json.dump(com_log_dict, json_file, indent = 2)
    
    # if CoM calculation goes wrong then continue
    proc_log_crop = {k : v for (k, v) in proc_log.items() if "cropping" in k}
    if len(proc_log_crop) == 0:
        os.remove(ct_nrrd_path)
        os.remove(rt_nrrd_path)
        continue
    
    # check the cropped volumes have been exported as intended
    assert(np.sum(np.array(list(proc_log_crop.values()))) == 0)
    assert(os.path.exists(ct_nrrd_crop_path))
    assert(os.path.exists(rt_nrrd_crop_path))
    
    sitk_vol = sitk.ReadImage(ct_nrrd_crop_path)
    vol_crop = sitk.GetArrayFromImage(sitk_vol)

    sitk_seg = sitk.ReadImage(rt_nrrd_crop_path)
    seg_crop = sitk.GetArrayFromImage(sitk_seg)
    
    # export the cropped subvolume CoM slice (CT + RTSTRUCT) for quality control
    export_png_slice(input_volume = vol_crop,
                     input_segmask = seg_crop,
                     fig_out_path = os.path.join(pat_dir_path, pat + '_crop_CT_CoM.png'),
                     fig_dpi = 220,
                     lon_slice_idx = 75,
                     cor_slice_idx = 75,
                     sag_slice_idx = 75,
                     z_first = True)

---

### On Fostering Reproducibility: Logging the Processing Details?


In [ ]:
csv_out_name = 'nsclc-radiomics_preproc_details.csv'
dataset_csv_path = os.path.join(data_base_path, csv_out_name)


df_keys = ['PatientID',
           'path_to_ct_dir', 'ctStudyInstanceUID', 'ctSeriesInstanceUID',
           'path_to_rt_dir', 'rtstructStudyInstanceUID', 'rtstructSeriesInstanceUID',
           'path_to_seg_dir', 'segStudyInstanceUID', 'segSeriesInstanceUID',
           'rt_exported', 'orig_shape', '1mm_iso_shape', 'crop_shape', 'com_int', 'bbox']

data = {k : list() for k in df_keys}

det_df = pd.DataFrame(data = data, dtype = object)

In [ ]:
a = list()

for pat_num, pat in enumerate(subcohort_df.PatientID.values):

    print("\rProcessing patient '%s' (%d/%d)... "%(pat, pat_num + 1,
                                                   len(subcohort_df.PatientID)),
          end = '')
    
    pat_df = subcohort_df[subcohort_df['PatientID'] == pat]
            
    # init a dictionary with the same keys as "df_keys" to populate the latter
    pat_dict = dict()
    pat_dict["PatientID"] = pat

    pat_dir_path = os.path.join(preproc_data_path, pat)
    pat_json_path = os.path.join(pat_dir_path, pat + '_lookup_info.json')

    with open(pat_json_path, 'r') as json_file:
        lookup_dict = json.load(json_file)
    
    pat_dict["path_to_ct_dir"] = lookup_dict[pat]["path_to_ct_dir"]
    pat_dict["ctStudyInstanceUID"] = pat_df['ctStudyInstanceUID'].values[0]
    pat_dict["ctSeriesInstanceUID"] = pat_df['ctSeriesInstanceUID'].values[0]

    pat_dict["path_to_rt_dir"] = lookup_dict[pat]["path_to_rt_dir"]
    pat_dict["rtstructStudyInstanceUID"] = pat_df['rtstructStudyInstanceUID'].values[0]
    pat_dict["rtstructSeriesInstanceUID"] = pat_df['rtstructSeriesInstanceUID'].values[0]

    pat_dict["path_to_seg_dir"] = lookup_dict[pat]["path_to_seg_dir"]
    pat_dict["segStudyInstanceUID"] = pat_df['segStudyInstanceUID'].values[0]
    pat_dict["segSeriesInstanceUID"] = pat_df['segSeriesInstanceUID'].values[0]
    
    # ----------------------------------------
    
    # populate the "rt_exported" field
    rt_folder = os.path.join(pat_dir_path, pat  + '_whole_ct_rt')
    pat_dict['rt_exported'] = [f for f in os.listdir(rt_folder) if 'gtv-1' in f.lower()][0].split('.nrrd')[0]   
    if pat_dict['rt_exported'] != 'GTV-1':
        a.append(pat)
    
    # ----------------------------------------
    
    dicom_ct_path = lookup_dict[pat]["path_to_ct_dir"]
    
    dcm_file_path = os.path.join(dicom_ct_path,                 # parent folder
                                 os.listdir(dicom_ct_path)[0])  # *.dcm files

    dcm_file = pydicom.dcmread(dcm_file_path)
    n_dcm_files = len([f for f in os.listdir(dicom_ct_path) if '.dcm' in f])

    xy = int(float(dcm_file.Rows)*float(dcm_file.PixelSpacing[0]))
    z = int(float(dcm_file.SliceThickness)*float(n_dcm_files))
    
    orig_dcm_shape = (n_dcm_files, dcm_file.Columns, dcm_file.Rows)
    res_dcm_shape = (z, xy, xy)
    
    pat_dict['orig_shape'] = orig_dcm_shape
    pat_dict['1mm_iso_shape'] = res_dcm_shape
    
    # ----------------------------------------
     
    com_json_path = os.path.join(pat_dir_path, pat + '_com_log.json')
    
    try:
        with open(com_json_path, 'r') as json_file:
            com_dict = json.load(json_file)
            pat_dict['com_int'] = tuple(com_dict["com_int"])
    except:
        print('_com_log.json loading error;')
    
    # ----------------------------------------
    
    bbox_json_path = os.path.join(pat_dir_path, pat + '_crop_log.json')
    
    try:
        with open(bbox_json_path, 'r') as json_file:
            bbox_dict = json.load(json_file)
            pat_dict['bbox'] = bbox_dict
    except:
        print('_crop_log.json loading error;')
    
    # ----------------------------------------
    ct_res_crop_path = os.path.join(pat_dir_path, pat + '_ct_res_crop.nrrd')
    
    try:
        sitk_ct_res_crop = sitk.ReadImage(ct_res_crop_path)
        pat_dict['crop_shape'] = sitk_ct_res_crop.GetSize()
    except:
        print('_ct_res_crop.nrrd loading error;')
        
    # ----------------------------------------
    
    det_df = det_df.append(pat_dict, ignore_index = True)

det_df.to_csv(dataset_csv_path)

---
---

# <center> Data Exploration </center>


---
---

# <center> Data Processing </center>


Brief description of the model here (image from the paper, or description of the architecture, also from the paper)?

Then, load the model.

In [11]:
arch_json_path = "demo/architecture.json"
weights_path = "demo/weights.h5"

"""
 N.B. the warnings are due to the fact that the model was developed for
 Keras 1, and the config file has been converted in a Keras-2-compatible file

 nonetheless, Keras 2 uses different naming conventions/def.s, so in order to
 get rid of the warnings one should change all the layers def.s in the JSON file
"""

# load the model architecture from the config file, then load the model weights 
with open(arch_json_path, 'r') as json_file:
    model_json = json.load(json_file)  

model = keras.models.model_from_config(model_json)
model.summary()

model.load_weights(weights_path)

/tensorflow-1.15.2/python3.6/keras/engine/base_layer.py:1179: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(name="convolution3d_1", activation="linear", trainable=True, input_dtype="float32", batch_input_shape=[None, 50,..., activity_regularizer={'class_na..., kernel_size=(5, 5, 5), filters=64, strides=[1, 1, 1], padding="valid", data_format="channels_last", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None, use_bias=True)`
  return cls(**config)
/tensorflow-1.15.2/python3.6/keras/engine/base_layer.py:1179: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(gamma_regularizer=None, name="batchnormalization_1", epsilon=0.001, trainable=True, beta_regularizer=None, momentum=0.99, axis=-1)`
  return cls(**config)
/tensorflow-1.15.2/python3.6/keras/engine/base_layer.py:1179: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(trainable

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution3d_1 (Conv3D)     (None, 46, 46, 46, 64)    8064      
_________________________________________________________________
batchnormalization_1 (BatchN (None, 46, 46, 46, 64)    256       
_________________________________________________________________
leakyrelu_1 (LeakyReLU)      (None, 46, 46, 46, 64)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 46, 46, 64)    0         
_________________________________________________________________
convolution3d_2 (Conv3D)     (None, 44, 44, 44, 128)   221312    
_________________________________________________________________
batchnormalization_2 (BatchN (None, 44, 44, 44, 128)   512       
_________________________________________________________________
leakyrelu_2 (LeakyReLU)      (None, 44, 44, 44, 128)  

In [20]:
# sanity check on model weights + visualisation?
model.get_weights()

[array([[[[[ 0.00447307,  0.0014684 ,  0.019163  , ..., -0.00107624,
             0.00943342,  0.03815292]],
 
          [[ 0.01036488, -0.00345621, -0.01784903, ...,  0.00185058,
            -0.02311649, -0.02566344]],
 
          [[-0.04935366, -0.01826825, -0.03895129, ...,  0.04801387,
             0.03173821,  0.02201864]],
 
          [[-0.02288254,  0.02196151,  0.02095928, ..., -0.05797391,
             0.0151874 , -0.01288855]],
 
          [[ 0.00633066, -0.03498531, -0.00074094, ...,  0.04083853,
            -0.00681345,  0.01131012]]],
 
 
         [[[ 0.02165704,  0.02454131,  0.00081817, ..., -0.03628308,
            -0.01519061, -0.02403273]],
 
          [[ 0.00624497, -0.0028399 ,  0.04503923, ..., -0.00548586,
             0.04154683, -0.02083678]],
 
          [[-0.01180558,  0.04914569,  0.00127675, ..., -0.0401111 ,
            -0.05337088, -0.02955528]],
 
          [[ 0.03666344, -0.0125441 ,  0.02350312, ...,  0.01162748,
             0.004398  ,  0.01981995]],
